## git rebase -i 

HEAD~n = e <n>th generation ancestor of HEAD
    
Start it with the last commit you want to retain as-is:

git rebase -i <after-this-commit>
    
Example: 
    
git rebase -i HEAD~3
    
Va ouvrir un fichier texte où chaque ligne correspond à un commit de la séquence allant de HEAD~3 (exclu) à HEAD (inclus), du plus ancien (en haut) au plus récent, HEAD (en bas).
    
pick HEAD~2 Oldest message
pick HEAD~1 Older message
pick HEAD Latest message
    
Changer en 
pick HEAD~2 Oldest message
squash HEAD~1 Older message
squash HEAD Latest message
    
Sauvegarder et quitter. 
    
On va se faire présenter un nouvel éditeur de texte pour éditer le commit message du nouveau commit à créer. Idem, éditer, sauvegarder et quitter. Le nouveau commit aura pour parent HEAD~3.

    
  

## Refs

En git, une ref est simplement défini comme un nom (name) qui débute par le préfixe `refs/`, comme par exemple `refs/heads/master`). Une ref pointe soit directement vers un nom d'objet (object name), on parle alors de named ref, soit vers une autre ref et donc indirectement vers un nom d'objet. On parle dans ce dernier cas de ref symbolique (symbolic ref ou symref, ex: `HEAD`). Dans le premier cas, on entend par nom d'object l'identifiant unique de cet objet qui correspond à sa SHA-1 et qui prend concrètement la forme d'une string de 40 caractères hexadécimaux.

Remarque: il y a dans les faits peu de symrefs. Les principales sont `HEAD` et `refs/remotes/<remote-name>/HEAD`.

Remarque: En Git, une "revision" est un synonyme pour commit (https://git-scm.com/docs/gitglossary#Documentation/gitglossary.txt-aiddefrevisionarevision)

Remarque: La ref `refs/remotes/<remote-name>/HEAD` est une symref locale correspondant à une copie de la ref `HEAD` du *remote repository* `<remote-name>`. Bien qu'un remote repository ne puisse pas avoir de worktree, une ref `HEAD` peut toujours y exister, elle désigne alors la *default branch* du *repository*, i.e. la branche qui sera *checked out* automatiquement après l'avoir cloné. La *default branch* d'un *remote repository* est managées à l'aide de la commande [`git remote set-head`](https://git-scm.com/docs/git-remote#Documentation/git-remote.txt-emset-headem) et fait partie des informations retournées par `git remote show`.

Concrètement, le fichier texte contient:
* Pour une named ref, le SHA-1 (40 caractères hexadécimaux) de l'objet vers lequelle elle pointe.
* Pour une symbolic ref: une string de la forme `ref: refs/some/thing` avec `refs/some/thing` la ref vers laquelle la symref pointe.

Les refs sont concrètement des fichiers textes stockés dans le *repository*, dans le répertoire `$GIT_DIR`. 

Les refs ne sont donc qu'une collection de noms et forment donc un namespace. Ce namespace est hiérarchique: le namespace `refs/` se divise ainsi par exemple en `refs/heads/`, `refs/tags/`, `refs/remotes/`, etc. 
* Chaque sous-élément pouvant avoir lui-même une structure. 
* Chaque élément de la hiérarchie sert un objectif différent. Par exemple: 
    * Les namespaces `refs/heads/`, `refs/tags/`, `refs/remotes/` servent à représenter les branches locales, les tags et les remote repositories 
    * Le namespace `refs/heads/` sert à représenter
    
Remarques: 
* Sur `refs/heads`: Git distingue la branche qui est simplement une chaîne de commits, de la (named) ref qui pointe vers l'extrémité (tip) de celle-ci et qui est appelée head (ou head ref). Les heads correspondent ainsi à l'ensemble des named refs pointant vers des extrémités de branches. Il existe un abus de langage fréquent confondant les heads et les branches auxquelles elles se rapportent.
* Un tag est une named ref comme une head ref. Le premier se distingue de la seconde par le fait qu'il n'est pas updaté par la commande `git commit`.
* Les *remote-tracking branches* sont des refs dédiées au suivi de l'état des remote repositories. Ces refs suivent typiquement le pattern `/refs/remotes/<remote-name>/<branch-name>` (ex: `/refs/remotes/origin/develop`) et correspondent à la partie droite (destination) des refspecs configurées pour `git fetch`. Ces branches se distinguent des head refs dans le sens où elles ne sont pas destinées à être modifiées directement par l'utilisateur dans le local repository. Elles sont gérées et déplacées par Git à chaque opération de synchronisation avec un remote repository.

Remarque: Par définition, le namespace `refs/remotes` n'existe pas côté remote repository.

https://git-scm.com/docs/gitglossary#Documentation/gitglossary.txt-aiddefsymrefasymref
https://git-scm.com/docs/gitglossary#Documentation/gitglossary.txt-aiddeftagatag
https://git-scm.com/docs/gitglossary#Documentation/gitglossary.txt-aiddefremotetrackingbrancharemote-trackingbranch

Concrètement, à chaque ref correspond sur le file system un fichier texte contenant le nom de l'entité vers laquelle pointe la ref. La hiérarchie des namespaces est reflétée par l'arborescence des répertoires. Par exemple, à la ref `refs/heads/master` correspond un fichier texte `master` situé dans le répertoire `$GIT_DIR/refs/heads`.

La hiérarchisation des namespaces peut être étendue par l'utilisateur en s'appuyant sur le même pattern qu'utilisé par Git, i.e. la délimitation par des `/`. Par exemple, si j'appelle ma branche `feature/pierre/my-feature`, à cette ref correspondra un fichier texte `my-feature` dans le répertoire `$GIT_DIR/refs/heads/feature/pierre`. On a étendu la hiérarchisation du namespace `refs/heads/`.

A côté des refs "classiques" (branches, tags, *remote-tracking branches* par exemple) qui sont globales, i.e. partagées par tous les worktrees, on distingue: 
* Les per-worktree refs (actuellement `HEAD` et `refs/bisect`) ont de différent le simple fait qu'elles ne sont pas globales et dans le cas de `HEAD`, qu'elles ne font pas partie du namespace `refs/heads`.
* Les pseudo-refs qui ne sont ni des named refs ni des symrefs mais qui se comportent comme des refs du point de vue de `git rev-parse`.

Pseudo-refs A RETRAVAILLER -> refs non stockées dans `$GIT_DIR/refs` mais dans `$GIT_DIR` / n'appartenant pas au namespace `refs/`
Les pseudo-refs sont des refs spéciales (special-purposes) faisant l'objet d'un traitement particulier par Git. Elles se caractérisent par les faits:
* Leur nom est en majuscule.
* D'être stockées dans `$GIT_DIR` et non dans `$GIT_DIR/refs`. Dit autrement, leur nom ne suit pas le pattern `/refs/*`, elles ne font pas partie du namespace `/refs`.
* Que leur fichier commence par un SHA-1 (et donc pointent toujours vers un nom d'objet) suivi d'un espace et optionnellement des données aditionnelles. En particulier et contrairement aux per-worktree refs, les pseudo-refs ne peuvent pas être des symbolic refs et n'ont pas de reflogs.

Les pseudo-refs ont de refs le fait qu'elles se comportent comme des refs du point de vue de `git rev-parse`: on peut les lire comme des refs et en particulier avec `git rev-parse`. En revanche, elles en diffèrent dans le sens où elles ne peuvent pas être symboliques, avoir de reflogs et ne sont pas updatées comme le sont les refs (elles le sont par écriture directe dans leur fichier). 

https://git-scm.com/docs/gitglossary#Documentation/gitglossary.txt-aiddefpseudorefapseudoref

Par exemple: MERGE_HEAD, CHERRY_PICK_HEAD et FETCH_HEAD (?) sont des pseudo-refs. HEAD n'est pas une pseudo-ref car elle peut être symbolique, mais une per-worktree ref.

MERGE_HEAD, CHERRY_PICK_HEAD, REBASE_HEAD, ORIG_HEAD ne peuvent pas avoir de reflogs et donc se singularisent des refs de ce point de vue car elle sont by design short-lived ou intermitentes.
FETCH_HEAD, MERGE_HEAD peuvent rassembler plus d'un nom d'objet et donc se singularisent des refs. De même, si on référence plusieurs noms, on ne peut pas tenir de reflog.
Dans cette perspective, HEAD est spéciale et se distingue de ces pseudo-refs car 1) elle est permanente / long-lived et 2) ne référence qu'un unique objet à la fois (même indirectement). 

Per-worktree refs
Les per-worktree refs sont une catégorie de refs permettant de regrouper des refs spéciales qui peuvent être qui sont attachées à un work tree et non globales (par ex, les refs `/refs/heads/` ne sont pas liées à un work tree particulier). C'est le cas par exemple de `HEAD` et des refs `/refs/bisect/`.

https://git-scm.com/docs/gitglossary#Documentation/gitglossary.txt-aiddefperworktreerefaper-worktreeref

Remarque: Différence entre un worktree et un working tree. Un worktree consiste en un working tree et de metadonnées propres au worktree comme l'index, un per-worktree configuration file, des per-worktree refs (`HEAD`, `/refs/bisect`) et pseudo-refs (`MERGE_HEAD`, etc.). Un repository rassemble un ou plusieurs worktrees. Les métadonnées du repository se décomposent en métadonnées partagées par tous les worktrees et les métadonnées maintenues séparemment par worktree. Dit autrement, on n'a pas un working tree par repository: il n'y a pas d'adhérence entre repository et working tree, cela étant permis par l'introduction d'une entité intermédiaire: le worktree. 

Cf. notamment https://stackoverflow.com/a/59093747 et la doc de [`git worktree`](https://git-scm.com/docs/git-worktree). 

Refs spéciales: 
* `HEAD`. Cette ref pointe directement ou indirectement vers le commit sur lequel sont basés les changement du *working tree*, i.e. le commit courant.
* `ORIG_HEAD`: Le fichier associé à cette ref n'existe pas toujours. `ORIG_HEAD` n'est créé que par les commandes au comportement considéré comme potentiellement dangereux, considérées comme déplaçant `HEAD` assez radicalement. `ORIG_HEAD` contient la référence du dernier commit vers lequel pointait `HEAD` avant son déplacement. `ORIG_HEAD` permet ainsi de revenir en arrière au cours d'une opération "dangereuse". Par exemple:
    * `ORIG_HEAD` permet d'annuler un `git reset --hard <commit>`, la commande `git reset --hard ORIG_HEAD` permettant de restaurer l'état antérieur au premier `git reset --hard`. Cf. la doc de `git reset` pour d'autres exemples (ex: `git reset --merge ORIG_HEAD`).
    * Les commandes `git merge` et `git pull` stockent dans `ORIG_HEAD` le nom du commit de la branche courante avant exécution de la commande, i.e. le nom du tip commit de la branche vers laquelle on merge ("ours").
* `MERGE_HEAD`: Créée lors d'un merge et à la présentation de conflits, cette ref rassemble le nom du ou des commits qu'on tente de merger dans notre branche courante ("theirs").
* `CHERRY_PICK_HEAD`: Créée lors d'un cherry-pick et à la présentation de conflits, cette ref pointe vers le nom du commit qu'on est en train de *cherry pick* sur notre branche courante.
* `REBASE_HEAD`: Créée lors d'un rebase et à la présentation de conflits, cette ref pointe vers le commit dont l'application au commit courant a généré le conflit.
* `FETCH_HEAD`: Cette ref enregistre l'ensemble des noms des tip commits des remote heads (branches) fetchées lors de la dernière exécution de la commande `git fetch`.

Remarque: `MERGE_HEAD`, `REBASE_HEAD` et `CHERRY_PICK_HEAD` ne vivent en principe que le temps de l'exécution de la commande qui les a créées.

Remarque: La commande `git bisect` créé aussi un certain nombre de refs `$GIT_DIR` comme `BISECT_HEAD`, `BISECT_START`, etc.

https://git-scm.com/docs/gitrevisions#Documentation/gitrevisions.txt-emltrefnamegtemegemmasterememheadsmasterememrefsheadsmasterem

HEAD
La ref spéciale `HEAD` pointe indirectement (le plus souvent) ou directement (cas detached HEAD) vers le commit sur lequel sont basés les changements le working tree, i.e. vers le commit actuellement checked out, i.e. le commit courant. Le working tree correspond en effet au contenu du tree du commit vers lequel pointe HEAD et des modifications locales (local changes) non encore commitées. Il s'agit du commit sur lequel la commande `git commit` créerait un nouveau commit. Il s'agit du commit par rapport auquel `git status` ou `git diff --cached` effectuent leurs comparaisons. Le plus souvent, `HEAD` est une symref pointant vers une des head refs du repository. C'est en ce sens qu'on peut parler pour `HEAD` de "branche courante mais attention, `HEAD` n'est justement pas toujours la branche courante. Or, Git nous permet de checkout n'importe quel commit et pas seulement des branches. Dans ce dernier cas, `HEAD` est une named refs pointant directement vers le checkout out. On parle alors d'état de detached `HEAD`: `HEAD` n'est plus dans son état "normal", i.e. "attaché" à une head ref (i.e. une branche).

Remarque: `@` est un raccourci pour `HEAD` qui rend les expressions de *revisions* particulièrement compactes. Cf. https://git-scm.com/docs/gitrevisions#Documentation/gitrevisions.txt-emem

`FETCH_HEAD`

`FETCH_HEAD` est le fichier dans lequel `git fetch` écrit les noms des refs qui ont été fetchées ainsi que les noms des objets vers lequelles celles-ci pointent. Le contenu de `FETCH_HEAD` est aussi bien destiné à être utilisé par d'autres commandes Git que par des scripts tiers. Le contenu de `FETCH_HEAD` s'organise comme suit, en trois colonnes:

```
71f026561ddb57063681109aadd0de5bac26ada9                        branch 'some-branch' of <remote URL>
669980e32769626587c5f3c45334fb81e5f44c34        not-for-merge   branch 'some-other-branch' of <remote URL>
b858c89278ab1469c71340eef8cf38cc4ef03fed        not-for-merge   branch 'yet-some-other-branch' of <remote URL>
```

A chaque ligne de `FETCH_HEAD` correspond une ref fetchée. Les refs fetchées sont le plus souvent des branches et c'est l'hypothèse qu'on fait pour la suite: 
* La première colonne contient le nom (i.e. son SHA-1) de l'objet vers lequel pointe la branche fetchée (i.e. son commit de pointe).
* La seconde colonne prend la valeur `not-for-merge` sauf pour une branche qui correspond à la branche checked-out au moment de l'exécution de la commande `git fetch` si une valeur est définie pour `branch.<name>.merge` (i.e. qu'elle est une *tracking branch*). Sinon la refspec marquée pour être mergée correspond à la première *fetchée* (cf. documentation de [`branch.<name>.merge`](https://git-scm.com/docs/git-config#Documentation/git-config.txt-branchltnamegtmerge)). Quand le fichier est utilisé par `git merge` (i.e. `git merge FETCH_HEAD`), la branche pour laquelle ce champ est vide correspond à celle qui est mergée dans la branche courante. La commande `git merge FETCH_HEAD` est en particulier exécutée lors d'un `git pull`, la [documentation](https://git-scm.com/docs/git-pull) de la commande précisant que par défaut, `git pull` n'est qu'un raccourci pour l'exécution d'un `git fetch` suivi d'un `git merge FETCH_HEAD`. En théorie, plus d'une branche peut ne pas prendre la valeur `not-for-merge` mais cela nécessitera d'opter pour une *merge strategy* capable de traiter plus d'une head ref à la fois, i.e. la stratégie `octopus`.
* La troisième colonne donne pour chaque ref fetchée, le nom de son remote repository d'origine ainsi que le nom qu'elle porte dans celui-ci.

La ligne correspondant à la branche courante semble toujours être la première du fichier (si celle-ci faisait partie des head refs matchant le ou les *refspecs* passées à `git fetch`). La commande `git rev-parse FETCH_HEAD` semblant toujours retourner le nom du commit de la première ligne, il retourne ainsi le nom du *tip commit* de la branche courante.

https://stackoverflow.com/a/45543739 

#### Désambiguïsation des refnames abrégés
La commandes prenant en argument un ou plusieurs `<refname>` autorisent que ceux qui leur sont passés soient "abrégés" ou dit autrement, ne soient pas totalement qualifiés (*unqualified*). On peut par exemple passer `master` au lieu de `refs/heads/master`. Ces commandes recourent alors à une logique de résolution (*disambiguation*) visant à produire une refname qualifié (*qualified*) c'est-à-dire, sauf pour les refs spéciales, commençant par `refs/`. Cette logique est en 6 étapes, le résultat finalement étant celui de la première étape dont la recherche réussi: 
1. Est-ce que `$GIT_DIR/<refname>` existe ? 
2. Est-ce que `refs/<refname>` existe ?
3. Est-ce que (le tag) `refs/tags/<refname>` existe ?
4. Est-ce que (la branche) `refs/heads/<refname>` existe ?
5. Est-ce que (le *remote*) `refs/remotes/<refname>` existe ?
6. Est-ce que `refs/remotes/<refname>/HEAD` existe ?

On remarque que Git cherche parmi les tags **puis parmi les branches**. En spécifiant une *unqualified refname*, on prend aussi le risque que la *disambiguation* produise des résultats inattendus. Par exemple en retournant pour `master`, `/refs/tags/master` au lieu de `ref/heads/master`.

Remarques: 
* Un *refname* abrégé n'a pas forcément à être la dernière partie du qualified *refname*. Par exemple, l'*unqualified refname* `/heads/master` est tout aussi acceptatble que `master`.
* Certaines commandes ne passent pas par le procédé en 6 étapes du fait que leur usage résolve intrinsèquement toute ambiguïté. Par exemple, `git branch` va directement chercher à résoudre un *unqualified refname* dans `/refs/heads`. Cf. https://stackoverflow.com/a/50637676, footnote 4.

https://git-scm.com/docs/gitrevisions#Documentation/gitrevisions.txt-emltrefnamegtemegemmasterememheadsmasterememrefsheadsmasterem

#### Refspecs
Une refspec est une formatted string utilisée par les commandes `git fetch` et `git push` pour spécifier avec quelle ref de la source updater quelle ref de la destination. Dans le cas de `git fetch`, la source correspond au remote repository et la destination le local repository. C'est logiquement l'inverse pour `git push`.

Le format d'une refspec est le suivant: `+<src>:<dst>` où:
* Le `+` est un paramètre optionnel signalant si l'update doit être forcée ou non dans le cas où celle-ci ne serait pas *fast-forward*. La présence du `+` est globalement équivalent à ajouter l'option `--force`.
* `<src>` est la ref de la source
* `<dst>` est la ref de la destination qu'on cherche à updater avec `<src>`

Dans le cas de `git push`: https://git-scm.com/docs/git-push#Documentation/git-push.txt-ltrefspecgt823082
* `<src>` peut ne pas être une ref mais n'importe quelle expression résolvant à un SHA-1 (ex: `master~4`), <src> en revanche doit logiquement être une ref.
* `<dst>` est optionnel, i.e. la partie `:<dst>` peut être omise (ex: `git push origin refs/heads/develop`), la ref de destination à updater sera soit celle donnée par la refspec `remote.<repository>.push` si celle-ci donne une ref à updater pour `<src>`, sinon on update la même ref que `<src>`.
* Si `<dst>` ne commence pas par `/refs` alors Git va essayer d'inférer la forme qualifiée de `<dst>` basée si l'ambiguïté de `<dst>` et le type de `<src>`:
    * Si `<dst>` ne commence pas par `refs/` mais n'est pas ambigu (ex: `heads/develop`) alors on *push* vers cette destination.
    * Sinon, et si la résolution de `<src>` produit une ref de `/refs/heads` ou `/refs/tag`, alors on ajoute ce préfixe à `<dst>`.
* Tout push vers `refs/tags/*` interdit (sauf force) les updates.
* Toutes les règles interdisant des updates peuvent être overridées avec l'ajout du `+` à la refspec ou de l'option `--force` à la commande sauf si ces mêmes forçages sont interdits par la configuration ou des hooks du *remote repository*.
* Ne pas préciser `<src>`, i.e. passer `:<dst>`, permet de supprimer `<dst>` du *remote repository*. Les suppressions sont toujours acceptées (on a notamment pas besoin de les forcer) sauf si explicitement interdites par la configuration ou des hooks du *remote repository*.
* Le edge case `:` (et `+:`) commande d'updater toutes les branches qui sont présentes sous le même nom à la fois sur le repository d'origine et de destination (*matching branches*).

Exemples: https://git-scm.com/docs/git-push#_examples
* `git push origin master`: Git développe (*expand*) automatiquement `master` en `refs/heads/master:ref/heads/master`. La refspec développée étant obtenue après désambiguïsation de `master` (supposée par défaut correspondre à `<src>`). On suppose que le résultat de cette désambiguïsation est `refs/heads/master`. On suppose que la config `remote.origin.push` n'est pas set, la partie `<dst>` est alors prise égale à la source.

Optionnalité de <src>/<dst>
Les updates fast-forward sont-elles toujours réalisées ?




Specify what destination ref to update with what source object. The format of a <refspec> parameter is an optional plus +, followed by the source object <src>, followed by a colon :, followed by the destination ref <dst>.


https://git-scm.com/book/en/v2/Git-Internals-The-Refspec

https://git-scm.com/docs/git-fetch
https://git-scm.com/docs/git-push

https://git.seveas.net/the-meaning-of-refs-and-refspecs.html
https://git-scm.com/docs/gitglossary#Documentation/gitglossary.txt-aiddefrefspecarefspec
https://git-scm.com/docs/git-fetch#Documentation/git-fetch.txt-ltrefspecgt
https://stackoverflow.com/a/50637676

#### Remote tracking branches et configs/commandes associées
Liens:
* La section [Tracking branches](https://git-scm.com/book/en/v2/Git-Branching-Remote-Branches) du chapitre Git Branching - Remote Branches du Git Book.
* https://stackoverflow.com/a/37770744

Une *tracking branch* est un sous-type de branches locales qui a une relation directe à une *remote branch* appelée *upstream branch*. Une *tracking branch* suit (*tracks*) une *upstream branch*. L'*upstream branch* est une propriété (i.e. une config) **optionnelle** de la branche locale. Une branche peut avoir au plus une *upstream branch*. La commande principalement impactée par l'existence d'une *upstream branch* est `git pull` mais d'autres commandes sont également impactées.

Une autre fonctionnalité associée au fait d'avoir une *tracking branch* pour sa branche locale est qu'on peut dès lors avoir accès via [`git status`](https://git-scm.com/docs/git-branch#Documentation/git-branch.txt--t) (mais aussi `git branch -v` et `-vv`) aux différences en termes de *commits* entre les deux. Par exemple, si on est *checked out* sur `master` qui est *tracking branch* d'`origin/master`, un `git status` nous renseignera si on a des *commits* à *pusher* (on est en avance, *ahead*) et/ou à intégrer (on est en retard, *behind*).

Remarques: 
* Git fournit le *shorcut* `@{upstream}` ou `@{u}` pour référencer l'*upstream branch* de la branche courante. Par exemple, si `master` est *tracking branch* d'`origin/master` alors `git merge @{u}` est équivalent à `git merge origin/master`. 
* Pour lister les branches locales et faire apparaitre leurs éventuelles *upstream branches*, utiliser `git branch -vv`. Pour chaque branche, l'*upstream branch* apparaît entre `[]` avec leur éventuelle différence de position.

L'*upstream branch* est définie par deux configs `branch.<branch_name>.remote` et `branch.<branch_name>.merge`. Une *upstream branch* est donc davantage un ensemble de configurations qu'une "vraie" branche, c'est pourquoi on parle aussi plus justement de *tracking configuration* (on rencontre aussi *upstream configuration*). Une telle définition doit se comprendre par le fait qu'une *upstream branch* vise à autoriser une utilisation sans arguments des commandes `git fetch` / `git pull` / `git push` (A CONFIRMER POUR FETCH ET PUSH), i.e. on ne precise pas de *refspec* et eventuellement pas de *remote repository*. Ainsi:
* [`branch.<name>.remote`](https://git-scm.com/docs/git-config#Documentation/git-config.txt-branchltnamegtmerge) dit de quel (resp. vers quel) *remote repository* `git fetch` (resp. `git push`) doit récupérer (resp. envoyer) des données quand on est *checked out* sur `<name>` et qu'on a utilisé ces commandes sans préciser de *repository*. Noter que dans le cas d'un push sans précision de remote, ce setting est overridé par `remote.pushDefault`, lui même overridé par `branch.<name>.pushRemote`.
* [`branch.<name>.merge`](https://git-scm.com/docs/git-config#Documentation/git-config.txt-branchltnamegtremote) dit à `git pull` quelle branche intégrer dans la branche courante. Cette config est en effet utilisée par `git fetch` pour marquer la branche destinée à être mergée dans `FETCH_HEAD` (la valeur de la config devant matcher une ref du remote duquel on fetch). Sans cela, `git pull` merge par défaut la première refspec de `FETCH_HEAD` ("*the first refspec fetched*"). En cas de valeurs multiples, `git pull` un *octopus merge*. **Attention**: la ref renseignée dans cette configuration correspond à celle de la branche dans le *remote repository* (ex: `refs/heads/master`), le *mapping* avec la *remote-tracking branch* située sur le *local repository* est fait via les *refspecs* de `remote.<remote>.fetch`.

Les *upstream branches* sont essentiellement une fonctionnalité pour `git pull`, elles sont d'ailleurs listées dans la section `Local branches configured for 'git pull'` dans la sortie de la commande `git remote show <remote>`. 

Remarque: La commande `git remote show <remote>` fait aussi apparaître une section `Local refs configured for 'git push'`. Toutefois et contrairement à `git pull` et à ce que cette liste laisserait penser, il n'existe pas de *mapping* ou d'équivalent des *tracking branches* qui donnerait pour chaque *local ref*, la *ref* vers laquelle est serait *pushée* sur le *remote repository*. Cette section ne donne pour chaque *ref* locale celle du *remote repository* vers laquelle Git **essayerait** (*would attempt*: cf. cette [réponse SO](https://stackoverflow.com/a/45531248)) de la *pusher*. Sans doute que Git doit notamment s'appuyer sur les *refspecs* de `remote.<remote-name>.push` si celle-ci est définie pour dresser cette liste.

La définition de la *tracking configuration* peut inclure une troisième configuration optionnelle `branch.<name>.rebase` qui dit quelle valeur passer à l'option `-r`/`--rebase` de `git pull`. Celle-ci peut prendre 4 valeurs (`false`, `true`, `merges` ou `m`, `interactive` ou `i`) et override la config `pull.rebase` qui donne la valeur par défaut de cette option.

Remarque: `branch.<name>.rebase` peut être définie pour une branche sans que celle-ci admette de *tracking configuration*. Dans ce cas, la configuration précise comment `git pull` doit updater la branche courante avec la branche fetchée.

Ainsi, si on est *checked out* sur une *tracking branch*, `git pull` sait automatiquement quelle branche de quel *repository* récupérer (*fetch*) et merger. De même, `git push` sait vers quelle branche de quel *repository* pusher.

Quand on créé une nouvelle branche locale `<branch-name>`, on utilise la commande `git branch <branch-name> [<start-point>]` où `<start-point>` est un commit, un tag ou une branche (i.e. une *head ref*, ce qui inclut les *remote-tracking* branches), ce dernier argument pouvant être omis, la valeur par défaut `HEAD` étant alors utilisée. 

Le définition automatique d'une *upstream branch* (appelée aussi *tracking configuration*) à la création de `<branch-name>` dépend de la config [`branch.autoSetupMerge`](https://git-scm.com/docs/git-config#Documentation/git-config.txt-branchautoSetupMerge) et de l'option [`--track`](https://git-scm.com/docs/git-branch#Documentation/git-branch.txt--t) ou `--no-track` éventuellement passée à `git branch`.

Remarque: La définition automatique d'une *upstream branch* peut occurir avec `git branch` mais aussi avec `git checkout` et `git switch` (cf. ci-dessous).

La config `branch.autoSetupMerge` peut prendre 4 valeurs, chacune étant associée à un comportement:
* `false`: Il n'y a jamais de definition automatique d'*upstream branch*. Equivalent à toujours exécuter `git branch` avec `--no-track`. 
* `true`: Il n'y a de définition automatique que si `<start-point>` est une *remote-tracking branch*. **Il s'agit du comportement par défaut.**. Dans ce cas, c'est la *remote-tracking branch* elle-même qui est définie comme *upstream branch* de `<branch-name>`. Equivalent à exécuter `git branch` avec `--track=direct`.
* `always`: Il y a toujours définition automatique, que `<start-point>` soit une *local branch* ou une *remote-tracking branch*. Comme pour `true`, `<start-point>` est alors directement prise comme *upstream branch* pour `<branch-name>` (i.e. équivalent à exécuter `git branch` avec `--track=direct`). Dans le cas où <start-point> serait une *local branch*, `branch.<branch_name>.remote` est définie à `.`.
* `inherit`: Il n'y a de définition automatique que si `<start-point>` admet une *tracking configuration*. La *tracking configuration* établie pour `<branch-name>` est alors la copie de celle de `<start-point>` (i.e. équivalent à exécuter `git branch` avec `--track=inherit`).

Globalement, la configuration `branch.autoSetupMerge` contrôle si l'appel à `git branch`/`git checkout -b`/`git checkout -c` implique automatiquement l'option `--track` et avec quelle valeur.

La config `branch.autoSetupMerge` est overridée en cas d'utilisation des options `--track` ou `--no-track` de `git branch`:
* Avec l'option `-t`, `--track`, `--track=direct`, `<start-point>` est directement prise comme *upstream branch* pour `<branch-name>`.
* Avec l'option `--track=inherit`, la *tracking configuration* de `<branch-name>` est prise comme copie de celle de `<start-point>`.
* Avec l'option `--no-track`, aucune *tracking configuration* n'est créée quelle que soit la valeur de `branch.autoSetupMerge`.

Lors de la création automatique d'une *tracking* configuration, la configuration `branch.<name>.rebase` peut éventuellement être fixée automatiquement à `true` (et seulement à cette valeur, bien que 4 valeurs différentes soient possibles) suivant la valeur prise par la configuration [`branch.autoSetupRebase`](https://git-scm.com/docs/git-config#Documentation/git-config.txt-branchautoSetupRebase). Celle-ci peut prendre 4 valeurs possibles, chacune étant associée à un comportement: 
* `never`: La configuration `branch.<name>.rebase` n'est jamais fixée automatiquement à `true`.
* `local`: La configuration `branch.<name>.rebase` n'est fixée automatiquement à `true` que si l'*upstream branch* est locale.
* `remote` : La configuration `branch.<name>.rebase` n'est fixée automatiquement à `true` que si l'*upstream branch* est une *remote tracking branch*.
* `always`: La configuration `branch.<name>.rebase` est toujours fixée automatiquement à `true`.

Remarque: À propos de `branch.autoSetupMerge` et `branch.autoSetupRebase`, voir aussi [cette réponse](https://stackoverflow.com/a/22147540) Stack Overflow.

Une *tracking configuration* pour `<branch-name>` peut être définie indépendamment de la création de la branche. On utilise alors la commande `git branch --set-upstream-to=<upstream-branch-name> [<branch-name>]` ou de façon équivalente `git branch -u <upstream> [<branch-name>]`, `<upstream-branch-name>` étant une *remote-tracking branch* qui doit déjà exister sur le *local repository*. C'est par exemple le cas si on veut associer notre branche locale à une *remote-tracking branch* qu'on vient de *fetch* ou créée par un *push* antérieur. De la même manière, créer une nouvelle branche ne crée pas automatiquement de *tracking branch* (la *remote-tracking branch* éventuellement associée et pouvant faire office d'*upstream branch* n'existant par exemple, pas encore). Si une *tracking configuration* est déjà définie, celle-ci est alors écrasée. On peut également supprimer une *tracking configuration* existante avec `git branch --unset-upstream [<branch-name>]`. En cas d'omission de `<branch-name>`, la valeur prise par défaut est `HEAD`.

Contrairement à ce qu'on pourrait penser, bien que lors du premier push, `git push` créé une *remote tracking branch*, celle-ci n'est pas définie comme *upstream branch* de la branche poussée. On peut en revanche demander à `git push` de créer une *tracking configuration* à partir de la remote-tracking branch au premier comme lors de n'importe quel *push* à l'aide de son option [`-u`/`--set-upstream`](https://git-scm.com/docs/git-push#Documentation/git-push.txt--u), `git push` associe alors à chaque branche locale déjà à jour ou pour lesquelles l'opération a réussi, la branche vers laquelle elle a été poussée comme *tracking branch*. En général, en l'absence de préoccupation pour les *tracking configurations*, les seules branches locales en présentant sont celles qui ont été créées via *checkout* d'une *remote tracking branch* ce qui inclut notamment toujours la *default branch* du *repository*.

La définition d'une *upstream branch* peut aussi se faire quand la branche est créée via `git checkout -b` ou `git switch -c`. En particulier, `git checkout` et `git switch` fournissent les options `-t`/`--track` et `--no-track` qui sont finalement passées à `git branch`. Ainsi `git checkout -b <branch-name> --track <start-point>` inclut l'exécution de `git branch <branch-name> --track <start-point> ` (idem pour `git switch -c`).

Comme créer une branche depuis une *remote tracking branch* est une opération particulièrement courante, `git checkout` et `git switch` proposent deux *shortcuts* permettant de créer une branche locale avec *tracking configuration* à partir d'une *remote tracking branch*:
* On *checkout* directement la *remote tracking branch* avec l'option `--track` mais en l'absence de l'option `-b` (`git checkout`) ou `-c` (`git switch`). Par exemple, dans `git checkout --track <remote>/<branch-name>`, Git va déduire de `<remote>/<branch-name>` que le nom de branche locale à créer est `<branch-name>` et va lui créer une *tracking configuration* comme si on avait exécuté `git branch <branch-name> --track <remote>/<branch-name>`. Cf. la documentation de `--track` de `git checkout` et `git switch`.
* Encore plus simple, on peut directement *checkout* la branch (qui n'existe pas encore) sans préciser la *remote tracking branch* (sans utiliser les options `-b`/`-c`). Par exemple, dans le cas de `git checkout <branch-name>` (idem pour `git switch`), si 1) `<branch-name>` n'existe pas en local 2) il existe une branche `<branch-name>` dans exactement un *remote* appelé `<remote>`, alors cette commande est équivalente à `git checkout -b <branch-name> --track <remote>/<branch-name>`.

Remarque - Lien avec git merge sans arguments - Si on exécute la commande `git merge` sans lui passer de ref, et si une *upstream branch* est définie pour la branche courante, alors c'est cette dernière qui est mergée dans la branche courante (`git merge` récupère le nom de la *remote* branche et du *remote repository* dans `branch.<current-branch>.remote` et `branch.<current-branch>.remote` respectivement puis se sert des *refspec* de `remote.<remote>.fetch` pour obtenir le nom de la *remote-tracking branch* à finalement *merger* et située en local). Ce comportement est contrôlé par la config [`merge.defaultToUpstream`](https://git-scm.com/docs/git-config#Documentation/git-config.txt-mergedefaultToUpstream), à `true` par défaut.

Remarque - Lien avec la suppression d'une branche - Une branche locale ne peut être effacée sans forcer (i.e. sans perte de commits) que si elle est mergée dans son *upstream branch* ou si celle-ci n'est pas définie, dans `HEAD`.


#### Reflog et pseudo-refs 
https://git-scm.com/docs/git-reflog

#### refs/stash, git stash, reflog
https://git.seveas.net/the-meaning-of-refs-and-refspecs.html see Stash section
https://stackoverflow.com/questions/20586009/how-to-recover-from-git-stash-save-all
https://stackoverflow.com/questions/20409853/git-stash-and-apply
https://git-scm.com/docs/git-stash

git stash créé en fait un commit qui est ajouté au reflog de la ref refs/stash (ou `stash` en abrégé). refs/stash n'est pas un namespace / une famille de refs une une ref unique qui pointe vers le dernier stash (commit) créé, i.e. stash{0}. Les éventuels différents stash correspondent à autant d'entrées du reflog de refs/stash et non à des refs distinctes d'un namespace refs/stash. La commande git stash list retourne en fait le reflog de refs/stash et sa sortie est très similaire à celle de git reflog stash (celle-ci donne en plus les commits names de chaque stash). 

#### git worktree 
https://www.youtube.com/results?search_query=git+worktree

#### git fetch --prune
Actually manages our namespace? What about CRUD operations on the local refs namespace ?
git fetch simple ne supprime pas en local les refs supprimées sur le distant (d'où le flag --prune), ne fait pas par défaut les mappings type 
 (none)     -> origin/refacto/remove-promo-effectiveness-folder


#### https://git-scm.com/docs/git-config#Documentation/git-config.txt-branchltnamegtmerge

#### checkout lastest branch https://marcgg.com/blog/2015/10/18/git-checkout-minus/
https://stackoverflow.com/questions/55660409/is-there-a-direct-way-to-determine-gits-n-branch-without-e-g-parsing-a-lo

#### @{upstream} @{u} @{push}
http://git-scm.com/docs/gitrevisions#Documentation/gitrevisions.txt-emltbranchnamegtupstreamemegemmasterupstreamememuem

A confirmer mais il semble qu'apres un git fetch simple:
* On retrouve dans FETCH_HEAD toutes les branches du repo distant
* On ne voit dans la sortie de la commande que les updates, i.e. les branches ayant changé depuis le dernier fetch
* On ne voit ni dans FETCH_HEAD, ni dans la sortie les branches supprimées du repo. Normal. Par définition: FETCH_HEAD semble contenir que les branches présentes sur le repo. Il ne contient pas d'info sur les changements (ie des updates, des suppressions). Les updates effectuées par défaut n'incluent pas les suppressions (car non inclus dans la refspec par défaut ?).

----

A name that begins with refs/ (e.g. refs/heads/master) that points to an object name or another ref (the latter is called a symbolic ref). For convenience, a ref can sometimes be abbreviated when used as an argument to a Git command; see gitrevisions[7] for details. Refs are stored in the repository.

The ref namespace is hierarchical. Different subhierarchies are used for different purposes (e.g. the refs/heads/ hierarchy is used to represent local branches).

There are a few special-purpose refs that do not begin with refs/. The most notable example is HEAD.

https://stackoverflow.com/questions/62527864/how-to-determine-which-parent-is-the-first-one-for-a-merge-commit-in-git
https://www.atlassian.com/git/tutorials/advanced-overview
https://stackoverflow.com/questions/38864884/why-does-this-cherry-pick-have-a-conflict
https://stackoverflow.com/questions/11835948/git-cherry-pick-vs-rebase/11837630
https://stackoverflow.com/questions/35123108/cherry-pick-and-squash-a-range-of-commits-into-a-subdirectory-or-subtree
https://stackoverflow.com/questions/28189707/whats-the-difference-between-git-merge-squash-and-git-cherry-pick/28193740
https://stackoverflow.com/questions/3107789/using-git-how-do-you-reset-the-working-tree-local-file-system-state-to-the-st
* git checkout :/ see https://git-scm.com/docs/gitglossary#Documentation/gitglossary.txt-aiddefpathspecapathspec
* https://git-scm.com/docs/git-checkout-index
* `git checkout .`: brittle since dep on current working directory. `git checkout :/` more robust since indep from current wd.

git reset --soft us_fork_point pour bouger develop_usa bad idea -> on bouge la branche vers laquelle pointe HEAD et donc develop_usa !! -> problème réglé si on fait un git checkout --detach avant le git reset (https://stackoverflow.com/a/15993574 - dans ce cas on bouge vers une branche et non un commit, on rajoute une dernière commande qui permet de sortir de l'état de detached HEAD et de faire pointer à nouveau HEAD vers la branche cible et non juste le même commit qu'elle).

Cherry pick a range of commits: https://www.tollmanz.com/git-cherry-pick-range/

Merge: 
* La branche que je tire (ex: je suis develop, je tire topic) n'a pas un fichier que j'ai ajouté: comment je sais que ce fichier ne sera pas supprimé ? Quelle sont les règles que Git se donne ?

https://blog.plasticscm.com/2011/09/merge-recursive-strategy.html
https://blog.sogilis.com/posts/2015-05-12-demystifying-git-concepts/

Git looks for the merge base commit, which is the closest commit reachable through both parents.

## git commit --amend

## git diff 

* git diff A..B réalise le delta entre les commits A et B. Sur le sens de la différence, les différentes propositions sont équivalentes: 
    * A..B
    * Le résultat correspond à B "moins" A
    * Le résultat correspond au delta permettant de transformer A en B, d'aller de A vers B (A..B = A --> B).
    
## git push / pull
Remarque : Pour les commandes prenant un repository en argument, il est possible de passer le local repository courant, celui-ci étant désigné par `.`

Que se passe-t-il quand on ne passe pas d'arguments
Pusher / puller (surtout puller) sans être checkout
Un pull = un fetch mais pas un fetch de la seule branche mais de toutes les remotes tracking branches, un push repousse-t-il tout ?

https://stackoverflow.com/a/50637676
https://stackoverflow.com/a/17722977
https://stackoverflow.com/a/64205473
https://superuser.com/a/1382791

`+refs/heads/*:refs/remotes/origin/*` : permet notamment de fetch et d'updater les remote tracking branches de remotes branches qui ont été rebase.

## git push
*Remarque:* Si on utilise une forme complète (`git push <repository> <refspec>`), on a pas besoin d'être *checked out* sur la branche qu'on souhaire *pusher*. En revanche, les formes incomplètes finissent par *pusher* la branche courante (sauf ajustements de configuration), la branche sur laquelle ont est *checked out* a donc par définition un impact sur l'effet de la commande.

### Formes incomplètes
## `git push origin`
Quand la commande ne précise pas quoi *pusher*, i.e. absence de `<refspec>` (`git push origin <refspec>`) ou de *flags* comme `--all`, `--mirror` ou `--tags`, elle va chercher pour savoir quoi *pusher*, à récupérer le *default refspec*.
        
La récupération du *default refspec* est contrôlé par:
* La config `remote.<name>.push` du *remote* (ex: `remote.origin.push`) qui précise la *default refspec* du *remote* (ex: `HEAD`) ou si non définie,
* La config [`push.default`](https://git-scm.com/docs/git-config#Documentation/git-config.txt-pushdefault). Celle-ci prend sa valeur parmi un nombre fini d'options (ex: `current`, `upstream`, `simple`, etc.). Si non définie, sa valeur implicite est (pour Git 2.x) `simple`. La stratégie `simple` consiste simplement à pousser la branche courante vers le *remote* avec le même nom.

Ainsi sans modification de configuration, la commande `git push origin` aboutit à pusher la branche courante vers le *remote* `origin`.

## `git push`
Quand la commande ne précise pas vers quel *remote* `<repository>` *pusher* (`git push` au lieu de `git push origin` par exemple), elle récupère le *remote* de destination des configurations suivantes:
* [`branch.<current_branch_name>.pushRemote`](https://git-scm.com/docs/git-config#Documentation/git-config.txt-branchltnamegtpushRemote),
* Si la précédente n'est pas définie, `remote.pushDefault`,
* Si la précédente n'est pas définie, `branch.<current_branch_name>.remote` (définie si `<current_branch>` est une *tracking branch*),
* Si la précédente n'est pas définie, utilise `origin`.

https://git-scm.com/docs/git-push#_description
https://git-scm.com/docs/git-config#Documentation/git-config.txt-remotepushDefault

En résumé, sans modification de la configuration, la commande `git push` revient à pusher la branche courante vers son *remote repository*.
 
Remarque: Consulter une config Git: `git config [--get] <config_name>`

## TODO

D'après l'aide de git config: 
branch.<name>.merge : Defines, together with branch.<name>.remote, the upstream branch for the given branch.

* origin/HEAD : default branch What is it ?
https://stackoverflow.com/a/4359327
https://stackoverflow.com/a/6838756
https://stackoverflow.com/questions/8839958/how-does-origin-head-get-set

git diff + git apply : merge conflicts ? C'est un 2-way merge donc pas sûr qu'on les voit. Cf option --3way de git apply pour voir comment ça fonctionne
Conflicts with cherry-pick : 3 way merge conflicts ? https://stackoverflow.com/a/10058070 https://stackoverflow.com/a/56053884

Stratégie:
* Faire converger absolument ?
* Résoudre tous les merge conflicts par bloc, git merge develop, tester, merge ?

En git, une même commande peut avoir différents modes de fonctionnement avec des logiques et des effets potentiellement assez différents. Ex: git checkout avec les fonctionnalités restore et switch ou `git reset [<tree-ish>] [--] <pathspec>...` vs. `git reset [<mode>][<commit>]`

Pour voir toutes les façon de spécifier une "révision" (*revision*, synonyme de *commit*), par exemple `HEAD~2`, voir [cette section](https://git-scm.com/docs/git-rev-parse#_specifying_revisions) de la documentation de la commande `git rev-parse`.

Remarque: git reset --soft

Exemple de use-case: si on est pas satisfait de là où on se trouve en termes de working tree et d'index mais également en terme d'historique de commits qui nous a amené là. Un use case est donc du cleaning d'historique. Par exemple `git reset --soft HEAD~3` + `git commit` permet de squash les trois derniers commits (`HEAD`, `HEAD~1` et `HEAD~2`) + les différences en staging dans un seul commit (qui a comme parent `HEAD~3`). Les anciens commits devenant unreachable, ils sont de facto supprimés. `git commit --amend` qui permet de revenir sur le dernier commit (celui vers lequel pointe `HEAD`) peut être vu comme l'enchainement `git reset --soft HEAD^` (on déplace la branche et `HEAD` sur l'avant dernier commit, le working tree et l'index reflétant `HEAD`), modifications du working tree / index qu'on aurait voulu y inclure, `git commit -c ORIG_HEAD` qui commit avec les mêmes métadonnées que le commit vers lequel pointait `HEAD` au début de l'opération. `ORIG_HEAD` est une référence à l'état précédent de `HEAD`. Cette référence n'est pas tout le temps set, elle ne l'est que par certaines commandes dites "dangereuses" dont l'utilisation peut impliquer la perte définitive de code (dont `git reset` fait partie).

https://gist.github.com/gadzhimari/4670b8b01cfc5e15b35d0a524bfc9b1c
https://git-scm.com/book/en/v2/Git-Tools-Reset-Demystified
https://stackoverflow.com/questions/5203535/practical-uses-of-git-reset-soft

Pourquoi `git checkout` s'appelle comme cela ?

Pour ces raisons historiques, ce terme faisant partie plus largement du vocabulaire des VCS et notamment de ceux qui existaient déjà avant Git. On trouve ainsi deux verbes dans la terminologie des VCS: *check in* et *check out*:
* *Check in, means to store into the VCS*.
* *Check out, means to extract from the VCS*.

Git merge: tous les fichiers sont ajoutés à l'index, même ceux avec des conflits. C'est juste qu'on ne peut pas commit avec des fichiers de statut U dans l'index. Git n'autorise pas non plus à faire un git chechout -- file d'un fichier unmerged (la commande va chercher la copie du fichier située dans l'index).

Use case intéressant: repusher une branche déjà pushée mais rebased, git refuse par défaut le push (normal: on est non fast forward) mais plutôt que de conseiller le force, demande de pull la branche distante. Normal : le pull va 1) récupérer les dernières infos éventuellement pushées par quelqu'un, on ne va donc pas perdre de code et 2) va merger la remote tracking branch dans la branche locale. L'arborescence en résultant devient automatiquement fast forward et on peut pusher sans crainte. Problème: l'historique du projet n'est pas aussi clean que ce qu'on s'imagine en rebasant, un push force aurait préservé une structure clean comme on se l'imagine. Implique un merge (qui n'est pas simplement fast forward vu qu'on a rebase en local): crainte du DS de réintroduire des changements.

## `git merge`

|Theirs\Ours|Unmodified|Modified (M)|Added (A)|Deleted (D)|
|---|:---:|:---:|:---:|:---:|
|Unmodified|File unmodified and <br> therefore identical on <br> both sides. <br> No conflict.|Modified by us only. <br> No conflict.|Added by us only. <br> No conflict.|Deleted by us only. <br> No conflict.|
|Modified (M)|Modified by them only. <br> No conflict.|Modified on both sides. <br> **May create a conflict.**|X|**Conflict**|
|Added (A)|Added by them only. <br> No conflict.|X|Added on both sides. <br> **May create a conflict.**|X|
|Deleted (D)|Deleted by them only. <br> No conflict.|**Conflict**|X|Deleted on both sides. <br> No conflict.|

Une croix (X) correspond à des cas impossibles, par exemple il y a une contradiction à ajouter de notre côté (*ours*) un fichier qu'ils (*theirs*) ont modifié et donc qui existait déjà au niveau du dernier ancêtre commun.

Remarques: 
* Ces status sont calculés en référence à l'ancêtre commun (*3-way merge*)(cf. `...` de `git diff`). Seulement comparer les pointes des branches à merger sans référence commune (*2-way comparison*) ne permet pas de mettre en évidence les conflits. Dans ce mode de calcul des status, il est en particulier possible qu'un même fichier peut être répertorié M+M (ou A+A) tout en étant parfaitement identique de chaque côté au moment du *merge*.
* L'ancêtre commun entre deux branches `A` et `B` peut s'obtenir avec `git merge-base A B`.
* Toutes les commandes utilisant les *merge mechanics* (`git merge`, `git rebase`, `git cherry-pick`, etc.) ajoutent directement tous les fichiers the `theirs` au *working tree* et à l'index sauf pour les fichiers présentant des conflits qui sont eux seulement ajoutés au *working tree*. Dans le cas où on utiliserait la commande `git reset` sur tout ou partie des fichiers de l'index: 
    * Le statut des fichiers *added by them only* passe à *untracked*.
    * Restaurer les fichiers *deleted by them only* exige un *checkout* de `HEAD`.
    
Un merge peut-il réintroduire du code supprimé par l'autre branche: Non. Si `ours` a supprimé du code mais que `theirs` a encore le bloc de code non supprimé et qu'il n'a pas été modifié, le produit du *merge* est simple à déterminer: le code est supprimé. Dans cet exemple, une mauvaise résolution de conflits (créés par le fait que l'ancien code a été modifié côté `theirs`) peut conduire à la réintroduction d'ancien code mais c'est une erreur humaine et non de Git. Un merge est une opération faussement asymétrique, la commande `git mege theirs` pourrait laisser penser que `theirs` a une priorité sur `ours` (notamment quand c'est une branche de référence comme `develop`) mais ce n'est pas le cas. Les branches `ours` et `theirs` jouent un rôle symétrique, équivalent dans la détermination du produit de leur réunion, de leur *merge*.

Remarque: Les rename peuvent provoquer des conflits DD https://stackoverflow.com/questions/43702944/reproducing-git-merge-conflict-dd

Récupérer une branche par un merge rebase (pas de commit à deux parents) + squash en même temps pour obtenir qu'un commit de merge ? -> Rebase interactive

cherrypick vs rebase https://stackoverflow.com/questions/11835948/git-cherry-pick-vs-rebase

Rebase/cherry pick 3 way merge behavior:
https://stackoverflow.com/a/53974633 !!!
https://stackoverflow.com/a/65264492

The reason the "ours" and "theirs" notions get swapped around during rebase is that rebase works by doing a series of cherry-picks, into an anonymous branch (detached HEAD mode). The target branch is the anonymous branch, and the merge-from branch is your original (pre-rebase) branch: so "--ours" means the anonymous one rebase is building while "--theirs" means "our branch being rebased".

## Comment réaliser un "merge partiel" ?
Dans le processus de *merge* de sa branche dans une branche cible, la résolution des conflits passe par un *merge* (ou un *rebase*, non exploré ici) de la branche cible dans la branche à merger. On s'intéresse ici à la possibilité de ne *merger* qu'une partie et non l'intégralité de la branche cible dans la branche à merger pour permettre notamment une résolution fractionnée des conflits. 

On veut par exemple ne résoudre que les conflits de la partie `matrix/historical_view` de l'arborescence présents entre les branches `develop_usa` (branche à merger) et `develop` (branche cible). L'opération peut être réalisée par la séquence de commandes suivante:

1. `git checkout develop_usa`
2. `git merge --no-commit --no-ff develop` ou simplement `git merge develop` si la présence de conflits est déjà connue.
3. `git reset -- ':!matrix/historical_view'`
4. `git checkout -- ':!matrix/historical_view'`
5. `git clean -fd`
6. Résoudre les conflits et ajouter les fichiers impactés à l'index avec `git add`
7. `git commit`

Une fois la procédure de merge enclenchée, il est possible à tout moment de l'arrêter et de revenir à l'état initial sans perte de code avec `git merge --abort`.

Remarque: Pour voir comment utiliser les Git `pathspec`s (commandes 3 et 4), voir notamment [cette partie](https://git-scm.com/docs/gitglossary) de la documentation de Git et [ce post de blog](https://css-tricks.com/git-pathspecs-and-how-to-use-them/).

### Explications

#### Commande 1
On se place sur la branche à merger, i.e. `develop_usa`. 

#### Commande 2 
On amorce la procédure de *merge* de `develop` dans `develop_usa` avec `git merge --no-commit --no-ff develop`. Tous les fichiers ne présentant pas de conflits sont ajoutés à l'index, notamment les fichiers ajoutés par `develop` seule. Seuls les fichiers présentant des conflits (*updated but unmerged*, de statut `U`) sont présent dans le *working tree* sans l'être dans l'index.

En présence avérée de conflits, `--no-commit` est finalement facultatif, le merge s'interrompant de lui même pour permettre à l'utilisateur de régler manuellement. L'argument sert plus généralement à interrompre le *merge* avant le *commit* afin de permettre à l'utilisateur de contrôler les modifications apportées par celui-ci.

L'argument `--no-ff` est également facultatif. Il nous permet d'être le plus largement couverts dans la perspective de pouvoir interrompre le merge avant le *commit* afin de pouvoir contrôler les modifications apportées. Les *merges fast-forwards* ne sont en effet pas interrompus par `--no-commit`. En imposant que ce type de *merge* soit rendu *non fast-forward*, on permet à `--no-commit` de les interrompre. 

A l'issu de cette étape, apparaissent dans la sortie de `git status`:
* Dans le *working tree* et dans l'index: l'ensemble des fichiers modifiés par le *merge* et ne présentant pas de conflits.
* Dans le *working tree* mais pas dans l'index: l'ensemble des fichiers présentant des conflits.

#### Commande 3
De façon générale, la commande `git reset` utilisée dans son mode `git reset [<tree-ish>] [--] <pathspec>...` *reset* les entrées de l'index/de la *staging area* à leur état dans `<tree-ish>` pour tous les chemins/fichiers matchant la ou les `<pathspec>`. 

En particulier `git reset -- <pathspec>` (`--` facultatif si une seule `<pathspec>`) est équivalent à `git reset HEAD -- <pathspec>` et a donc finalement pour effet d'*unstage* tous les fichiers matchant `<pathspec>`. Ainsi, `git reset -- <pathspec>` peut se voir comme l'opposé, la transformation inverse de `git add <pathspec>`.

Remarque: Pour les fichiers de l'index n'ayant pas de contrepartie dans `<tree-ish>`, ces fichiers seront préservés dans le *working tree* mais n'apparaissant alors ni dans `<tree-ish>`, ni dans l'index, ils sont logiquement listés comme *untracked* (status `?`) par `git status`. Dans notre cas, ces fichiers correspondent aux fichiers ajoutés par `develop` depuis la divergence.

Cette commande écrase donc la copie de l'index de chaque fichier matchant la `<pathspec> ':!matrix/historical_view'` (i.e. "tous les fichiers n'étant pas dans `matrix/historical view`") avec sa contrepartie dans `HEAD` (i.e. `develop_usa`). **On *unstage* tous les fichiers n'étant pas dans `matrix/historical view`.**

A l'issu de cette étape, apparaissent dans la sortie de `git status`:
* Dans le *working tree* et dans l'index: les fichiers de `matrix/historical_view` modifiés par le *merge* et ne présentant pas de conflits.
* Dans le *working tree* mais pas dans l'index: 
    * Les fichiers (*tracked*) présents dans les deux branches ou seulement dans `develop_usa` et n'appartenant pas à `matrix/historical_view` qu'ils présentent des conflits ou non.
    * Les fichiers (*tracked*) présents dans les deux branches ou seulement dans `develop_usa` de `matrix/historical_view` présentant des conflits.
    * Les fichiers ajoutés uniquement par `develop` n'appartenant pas à `matrix/historical_view` et désormais *untracked*.

#### Commande 4
De façon générale, la commande `git checkout` utilisée dans son mode `git checkout [<tree-ish>] [--] <pathspec>` écrase tous les chemins/fichiers du *working tree* **et** de l'index matchant `<pathspec>` avec leur contrepartie dans `<tree-ish>`. Si `<tree-ish>` est omis, c'est l'index (**et non `HEAD`**) qui est utilisé pour repeupler le *working tree*. Cette commande permet en particulier d'annuler les changements faits depuis le dernier `git add` pour les fichiers ciblés par la commande. Pour les fichiers *unstaged* et donc par définition identiques dans `HEAD` et l'index, cette commande a pour effet de réinitialiser leur copie du *working tree* avec sa contrepartie dans `HEAD`. Ainsi, `git checkout [<tree-ish>] [--] <pathspec>` est l'équivalent pour les fichiers *unstaged* d'un `git reset --hard` (commande qui ne peut s'appliquer qu'à des *commits* et non des fichiers). 

Remarques: 
* Voir aussi la commande `git restore` introduite par Git 2.23 (août 2019) dans l'idée de répartir les responsabilités de `git checkout` dans deux commandes (`git restore` et `git switch`) au lieu d'une. Dans cette perspective, on peut également juger que des commandes comme `git reset` font beaucoup de choses et pourraient être découpées en plusieurs commandes dans le futur.
* Le mode `git reset [<tree-ish>] [--] <pathspec>...` diffère de `git reset [<mode>][<commit>]`: dans ce dernier mode, `git reset` s'applique à des *commits* et non à des fichiers puisqu'elle consiste alors à simplement à déplacer la branche pointée par `HEAD` - et non juste à déplacer `HEAD` comme `git checkout` - et à modifier seulement l'index (`--mixed`), l'index et le *working tree* (`--hard`) ou aucun des deux (`--soft`).

Dans notre cas, cette commande écrase tous les fichiers n'étant pas dans `matrix/historical_view` précédemment *unstaged* avec leur contrepartie dans `HEAD` (i.e. `develop_usa`). En particulier:
* Les fichiers supprimés sans conflits par `develop` sont restaurés et réapparaissent dans le *working tree*.
* Les fichiers qui présentaient des conflits mais qui ne matchaient pas la *pathspec* `':!matrix/historical_view'` sont restauré à leur état dans `HEAD`.
* Les fichiers devenus *untracked* à l'issu de l'étape précédente sont préservés et sont l'objet de la commande suivante.
* A l'issu de cette étape, apparaissent dans la sortie de `git status`:
    * Dans le *working tree* et dans l'index: les fichiers de `matrix/historical_view` modifiés par le *merge* et ne présentant pas de conflits.
    * Dans le *working tree* mais pas dans l'index:
        * Les fichiers (*tracked*) de `matrix/historical_view` présentant des conflits.
        * Les fichiers devenus *untracked* à l'issu du `git reset` de l'étape précédente. 

#### Commande 5, 6 & 7
Cette commande va supprimer de façon certaine (`-f`, on force) et récursive (`-d`) l'ensemble des fichiers *untracked* du *working tree* (fichiers ciblés par défaut par `git clean`). Cette commande supprimant définitivement les fichiers du *working tree* ont peut préalablement réaliser un *dry run* avec `git clean -n` pour contrôler la les fichiers ciblés.

A l'issu de cette étape, apparaissent dans la sortie de `git status`:
    * Dans le *working tree* et dans l'index: les fichiers de `matrix/historical_view` modifiés par le *merge* et ne présentant pas de conflits.
    * Dans le *working tree* mais pas dans l'index: les fichiers (*tracked*) de `matrix/historical_view` présentant des conflits.
    
On peut dès lors résoudre normalement les conflits, ajouter les fichiers pour lesquels les conflits sont résolus à l'index avec `git add` (commande 6), puis une fois le travail de résolution des conflits terminé, *commit* l'ensemble des changements (commande 7).

## Comment *cherry-pick* les modification d'un seul fichier ou d'un seul groupe de fichiers ?
Contrairement à ce qu'on pourrait penser, la commande `git cherry-pick` n'est pas capable de réaliser cette tâche sauf si les modification à transposer on été introduites dans des *commits* dédiés. La commande `git cherry-pick` n'est capable d'appliquer les changements introduit par un ou plusieurs *commits*: si le ou les *commits* à *cherry-pick* ont impacté d'autres fichiers que ceux dont on souhaite transposer les modifications, ces modifications indésirables feront aussi partie l'opération.

On propose deux méthodes différentes permettant de réaliser l'opération et présentées de la plus recommandée à la moins recommandée. On se repose sur l'exemple suivant: on veut appliquer à `develop` (ou à une branche partant de celle-ci et destinée à s'y intégrer) les modifications introduites par `develop_usa` sur le fichier `configs/usa/config.yml`.

* **Méthode 1:** 

```
git diff develop_usa -- configs/usa/config.yml | git apply -R
```

Ou de façon équivalente: 

```
git diff develop..develop_usa -- configs/usa/config.yml | git apply
```

Pour lequel on calcule le patch directement "dans le bon sens" (`develop_usa` "moins" `develop`, exprimé par `develop..develop_usa`, les `..` sont facultatifs) ce qui évite d'avoir à l'inverser avec l'argument `-R` de `git apply`. Cette commande est également plus sûre: le point de référence est bien `develop` et plus le *working tree*.

* **Méthode 2:** 

```
git checkout develop_usa -- configs/usa/config.yml
```

Remarques:
* Le double tiret `--` permet de séparer la partie commande + options de la liste de chemins de fichiers. Dans le cas à un seul fichier comme ici, il est facultatif.
* Ces commandes peuvent s'appliquer à autant de fichier qu'on le souhaire, il suffit de séparer leurs chemins par des espaces.

Sur les (subtiles) différences entre les méthodes 1 et 2:
* **Le point de comparaison**: La méthode 2 va en fait écraser le `configs/usa/config.yml` du *working tree* par celui de `develop_usa` ce qui est équivalent à calculer le delta entre `develop_usa` et `HEAD`. La méthode 1 calcule le delta entre le *working tree* et `develop_usa` (des modifications de `configs/usa/config.yml` du côté de la destination avant l'opération peuvent donc avoir un impact sur le résultat).
* **A quoi sont appliqués les changements**: La méthode 1 n'applique les deltas qu'au *working tree* (et c'est pourquoi c'est la plus recommandée). Il faut donc faire un `git add configs/usa/config.yml` en plus pour pouvoir *commit*. La méthode 2 applique les deltas au *working tree* ET à l'index/la *staging area*. On donc directement *commit* juste après.

**Attention: cas des *merge conflicts***
* Option 3way de git apply ? Option --merge de git checkout (forme `git checkout [-f|--ours|--theirs|-m|--conflict=<style>] [<tree-ish>] --pathspec-from-file=<file> [--pathspec-file-nul]`) ?
* Ou alors, checker au préalable l'existence de merge conflicts pour les fichiers visés. Si avéré ne pas utiliser ces méthodes.

https://stackoverflow.com/questions/62018133/git-how-to-apply-a-conflicting-patch
http://tedfelix.com/software/git-conflict-resolution.html#git-apply
https://stackoverflow.com/questions/16190387/when-applying-a-patch-is-there-any-way-to-resolve-conflicts
https://github.com/phchan9/til/blob/master/git/how-to-apply-git-patch-gracefully.md

## Comment *cherry-pick* un *merge commit* ?
Afin de propager les changements introduits dans une branche à l'occasion d'un *merge* vers une autre branche, il peut être commode d'y *cherry-pick* le commit de *merge*. Suivant la *merge strategy* en vigueur, le *merge commit* n'a pas le même nombre de parents ce qui implique une utilisation différente de la commande `git cherry-pick`. Notamment: 
* Un *merge* "traditionnel" (appelé aussi *no fast-forward mege* ou *three-way merge*) crée un *merge commit* à deux parents.
* Un *squash merge* (comme en vigueur sur `develop`) crée un *merge commit* à un seul parent.

Dans le cas du *merge commit* crée par un *squash merge* (un parent), `git cherry-pick` s'applique "normalement", on a simplement à exécuter `git cherry-pick <commit>`. 

Dans le cas du *merge commit* à deux parents, il faut se souvenir qu'en Git et contrairement à d'autres VCS, un *commit* ne stocke pas un delta mais un *snapshot* complet du *working tree* (voir notamment le paragraphe [Snapshots, Not Differences](https://git-scm.com/book/en/v2/Getting-Started-What-is-Git%3F) de la documentation). Quand l'exécution d'une commande peut se représenter comme l'application d'un ou plusieurs deltas (ex: `git cherry-pick`, `git rebase`, etc.), ce ou ces deltas sont en fait calculés à l'exécution.

Dans le cas du *cherry-pick* d'un *commit* à plusieurs parents, Git ne sait pas a priori quel parent utiliser pour le calcul du delta. Il faut le lui préciser à l'aide du *flag* `-m`/`--mainline`. Cet argument prend comme valeur un entier correspondant au numéro du parent, `1` référençant le dernier commit de la branche vers laquelle on a mergé (`ours`), `2` celui de la branche qui a été mergé (`theirs`). Ex: `git cherry-pick -n 2 <commit>`

Remarque: La liste des parents d'un *commit* de *merge* est accessible via `git log`. Dans le cas d'un *commit* de *merge*, la sortie renvoyée par la commande va introduire une ligne commençant par `Merge:` juste avant la ligne `Author:` où sont listés les différents parents. Cette sortie est accessible via `git log -n 1 <commit>`. Dans le cas où on est *checked out* sur le *commit* à inspecter, on peut ne pas donner `<commit>`, la commande prendant `HEAD` par défaut. `git log <commit>` liste l'ensemble de l'historique de `<commit>` en partant de celui-ci. L'option `-n 1` permet de limiter la profondeur de l'historique renvoyé à 1, c'est à dire à `<commit>` lui-même.

## Comment lister les fichiers de `develop_usa` restant différents de leur contre-partie sur `develop` ?
Dans le but de suivre l'évolution de la reconvergence, il peut être utile de lister les fichiers de `develop_usa` restant différents de leur contre-partie sur `develop`. Les fichiers listés dans Azure DevOps quand on créé une PR de `develop_usa` vers `develop` n'est en effet pas exactement ce qu'on cherche: la liste des fichiers impactés présentée dans l'onglet Files de la PR est calculée en référence à l'ancêtre commun entre `develop_usa` et `develop` et non en référence à `develop`. Dit autrement, cette liste rassemble l'ensemble des fichiers impactés (ajoutés, supprimés, modifiés, renommés, etc.) sur `develop_usa` depuis sa création. En particulier, un fichier modifié sur `develop_usa` et mais finalement identique à sa contrepartie sur `develop` à un instant t apparaîtra toujours dans cette liste (car cette dernière n'est justement pas calculée en référence à `develop`).

La liste des fichiers modifiés (les fichiers seulemement ajoutés ou supprimés ne posent en fait pas de problèmes) sur `develop_usa`, ayant une contrepartie sur `develop` et non encore identiques à celle-ci peut être obtenue à l'aide de la commande suivante: 

```
comm -12 <(git diff develop...develop_usa --name-only --diff-filter=M) <(git diff develop..develop_usa --name-only --diff-filter=M)
```

A toutes fins utiles, on peut *pipe* cette liste à `wc -l` pour en obtenir la longueur ou à `grep` pour filtrage.

Remarque: Pour la liste des différents status possibles pour un fichier, cf. la doc de `git status`.

### Comment cela marche-t-il ?
A haut niveau: 
* `git diff develop...develop_usa --name-only --diff-filter=M` retourne la liste des fichiers (flag `--name-only`) seulement modifiés (`--diff-filter=M`, on ne s'intéresse pas aux fichiers ajoutés, supprimés ou renommés, ces derniers pouvant d'ailleurs se ramener aux deux cas précédents) calculée en référence à l'ancêtre commun entre `develop` et `develop_usa` (signalé par les `...`).  
* `git diff develop..develop_usa --name-only --no-renames` calcule la même liste mais en référence à `develop` (signalé par les `..`). En particulier tout fichier identique dans les deux branches n'apparaît pas dans la liste retournée par cette commande.
* `comm -12 <(command1) <(command2)` réalise l'intersection entre les listes retournées par `command1` et `command2`.

Remarques: 
* Attention à ce qu'il advient des `...` lors du copier-coller.
* La liste des fichiers visibles dans l'onglet Files d'Azure DevOps peut s'obtenir avec la commande: `git diff develop...develop_usa --name-only --no-renames`

https://git-scm.com/docs/gitrevisions#_specifying_ranges

### Comment lister les fichiers modifiés par `develop_usa` mais depuis supprimés sur `develop` ?
Il peut arriver que des fichiers aient été modifiés sur `develop_usa` mais que ces derniers aient aussi été supprimés de `develop` depuis. Ces cas sont problématiques car on risque de réintroduire du *dead code* dans `develop`. Toutefois, ils manifesteront par autant de *merge conflicts* qui résolus de façon adéquate permettent d'éviter cet écueil.

La liste de ces fichiers peut s'obtenir avec la commande suivante: 

```
comm -12 <(git diff develop...develop_usa --name-only --diff-filter=M) <(git diff develop_usa..develop --name-only --no-renames --diff-filter=D)
```

Remarque: L'ordre dans `git diff develop_usa..develop` importe car on ne s'intéresse qu'aux fichiers supprimés (*deleted*: `D`), opération qui dépend du point de vue duquel on se place contrairement à la modification (*modified*: `M`). Un fichier de status `D` dans `git diff develop_usa..develop` est vu comme `A` (*added*, ajouté) avec `git diff develop..develop_usa`.


## Comment lister les fichiers présentants des *merge conflicts* ?
Lister les fichiers présentant des *merge conflicts* demande d'initier le merge. En étant *checked out* sur `develop`:

```
git merge develop_usa --no-commit
```

Remarque: Le `--no-commit` n'est nécessaire que si on ne sait pas a priori si des conflits se présenteront.

La liste des fichiers présentant des conflits est produite par:

```
git diff --name-only --diff-filter=U
````

Remarque:
* `U` désigne le statut "*updated but unmerged*".
* `git diff` sans argument fait la différence entre le *working tree* et l'index/*staging area*. Tous les fichiers ne présentant pas de conflits sont déjà ajoutés à la *staging area* par `git merge`. Ainsi, les fichiers du *working tree* dont l'alter ego dans la *staging area* est différent correspondent donc aux fichiers présentant des conflits.

Total file count with status different than "unmodified" git diff develop...develop_usa --name-only
Total no Renames git diff develop...develop_usa --name-only --no-renames

Renames [R] git diff develop...develop_usa --name-only --diff-filter=R

Added [A] git diff develop...develop_usa --no-renames --name-only --diff-filter=A
 - Added on develop_usa only
 comm -23 <(git diff develop...develop_usa --name-only --no-renames --diff-filter=A) <(git diff develop_usa...develop --name-only --no-renames --diff-filter=A)
- Added on both sides
- With conflicts
 comm -12 <(git diff develop...develop_usa --name-only --no-renames --diff-filter=A) <(git diff --name-only  --no-renames --diff-filter=U)
- Without conflicts

Deleted [D] git diff develop...develop_usa --name-only --no-renames --diff-filter=D
- Deleted on develop_usa only
comm -23 <(git diff develop...develop_usa --name-only --no-renames --diff-filter=D) <(git diff develop_usa...develop --name-only --no-renames --diff-filter=D)
- With conflicts 
comm -12 <(git diff develop...develop_usa --name-only --no-renames --diff-filter=D) <(git diff develop_usa...develop --name-only --no-renames --diff-filter=M)
- Without conflicts
- Deleted on both sides


Modified [M] git diff develop...develop_usa --name-only --diff-filter=M
 - Modified on either side - Non identical
  comm -12 <(git diff develop...develop_usa --name-only --diff-filter=M) <(git diff develop_usa..develop --name-only --no-renames --diff-filter=M)
  - Conflicts - Deleted on develop
   comm -12 <(git diff develop...develop_usa --name-only --diff-filter=M) <(git diff develop_usa..develop --name-only --no-renames --diff-filter=D)
  - Conflicts - Both sides 
  
 
 
 - Deleted on develop side (-> conflict)
 comm -12 <(git diff develop...develop_usa --name-only --diff-filter=M) <(git diff develop_usa..develop --name-only --no-renames --diff-filter=D)
 - Identical on both sides
 
 Number of conflicts: git diff --name-only  --no-renames --diff-filter=U

comm -12 <(git diff develop...develop_usa --name-only --no-renames --diff-filter=A) <(git diff develop...develop_usa --name-only --no-renames --diff-filter=A)

comm -23 <(git diff develop...develop_usa --name-only --no-renames --diff-filter=D) <(git diff develop_usa...develop --name-only --no-renames --diff-filter=D)

comm -12 <(git diff develop...develop_usa --name-only --diff-filter=M) <(git diff --name-only --no-renames --diff-filter=U)

No conflicts, modified either side
comm -23 <(comm -12 <(git diff develop...develop_usa --name-only --diff-filter=M) <(git diff develop_usa..develop --name-only --no-renames --diff-filter=M)) <(git diff --name-only  --no-renames --diff-filter=U) | wc -l

comm -23 <(git diff develop_usa...develop --name-only --diff-filter=M) <(git diff develop...develop_usa --name-only --diff-filter=M)